In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import statsmodels.api as sm

In [2]:
total = pd.read_csv('../total_data.csv')
total.head()

,State,Electricity Price (cents/kWh),CO2 (lb/MWh),renew prod (1000MWh),nonrenew prod (1000MWh),production (1000MWh),Urban Population,Urban Population (%),avg annual temp (F),avg annual precip (in),avg annual sunlight (kJ/m^2),num_plants,num_renewable,num_nonrenewable,total area (mile^2),water percentage,total consumption,Renewable Policies
0,Maine,17.44,336.612,8794,4736,13530,526309.0,38.6,41.0,45.5,3815.0,196,170,26,"35,380",12.80%,35399,57
1,Massachusetts,21.27,851.739,8917,17462,26379,6416895.0,91.3,47.9,48.6,3944.0,632,555,78,"10,554",26.10%,131292,193
2,New Hampshire,21.07,302.928,2784,16224,19008,803420.0,58.3,43.8,47.9,3891.0,60,38,22,"9,349",4.20%,33311,76
3,Rhode Island,19.30,811.317,1684,7023,8707,999191.0,91.1,50.1,49.1,3989.0,100,86,14,"1,545",33.10%,53421,61
4,Vermont,16.99,35.627,2589,9,2598,225850.0,35.1,42.9,46.0,3826.0,118,105,13,"9,616",4.20%,113,116


In [3]:
ave_treatment = np.mean(total['Electricity Price (cents/kWh)'])
treatment = [0 if value <= ave_treatment else 1 for value in total['Electricity Price (cents/kWh)']]
total.insert(2, 'Treatment', treatment)
total['total area (mile^2)'] = total['total area (mile^2)'].str.replace(',', '').astype(int)
total['water percentage'] = total['water percentage'].str.replace('%', '').astype(float) / 100
total.head()

,State,Electricity Price (cents/kWh),Treatment,CO2 (lb/MWh),renew prod (1000MWh),nonrenew prod (1000MWh),production (1000MWh),Urban Population,Urban Population (%),avg annual temp (F),avg annual precip (in),avg annual sunlight (kJ/m^2),num_plants,num_renewable,num_nonrenewable,total area (mile^2),water percentage,total consumption,Renewable Policies
0,Maine,17.44,1,336.612,8794,4736,13530,526309.0,38.6,41.0,45.5,3815.0,196,170,26,35380,0.128,35399,57
1,Massachusetts,21.27,1,851.739,8917,17462,26379,6416895.0,91.3,47.9,48.6,3944.0,632,555,78,10554,0.261,131292,193
2,New Hampshire,21.07,1,302.928,2784,16224,19008,803420.0,58.3,43.8,47.9,3891.0,60,38,22,9349,0.042,33311,76
3,Rhode Island,19.30,1,811.317,1684,7023,8707,999191.0,91.1,50.1,49.1,3989.0,100,86,14,1545,0.331,53421,61
4,Vermont,16.99,1,35.627,2589,9,2598,225850.0,35.1,42.9,46.0,3826.0,118,105,13,9616,0.042,113,116


In [4]:
confounders = total.iloc[:, 4:]
confounders.head()

,renew prod (1000MWh),nonrenew prod (1000MWh),production (1000MWh),Urban Population,Urban Population (%),avg annual temp (F),avg annual precip (in),avg annual sunlight (kJ/m^2),num_plants,num_renewable,num_nonrenewable,total area (mile^2),water percentage,total consumption,Renewable Policies
0,8794,4736,13530,526309.0,38.6,41.0,45.5,3815.0,196,170,26,35380,0.128,35399,57
1,8917,17462,26379,6416895.0,91.3,47.9,48.6,3944.0,632,555,78,10554,0.261,131292,193
2,2784,16224,19008,803420.0,58.3,43.8,47.9,3891.0,60,38,22,9349,0.042,33311,76
3,1684,7023,8707,999191.0,91.1,50.1,49.1,3989.0,100,86,14,1545,0.331,53421,61
4,2589,9,2598,225850.0,35.1,42.9,46.0,3826.0,118,105,13,9616,0.042,113,116


# Vanilla Inverse Propensity Weighting 

## Peropensity scores via Logistic Regression

In [5]:
def calculate_propensity(data):
    if isinstance(data, pd.DataFrame):
        confounders = data.iloc[:, 4:]
    else:
        confounders = data
    log_reg = LogisticRegression(max_iter=10000)
    log_reg.fit(confounders, data['Treatment'])
    propensity_scores = log_reg.predict_proba(confounders)[:, 1] 
    return propensity_scores

propensity_scores = calculate_propensity(total)
propensity_scores

array([2.65790161e-01, 9.95579934e-01, 3.83036365e-01, 7.34255331e-01,
       5.59937711e-01, 9.91057245e-01, 9.99713405e-01, 6.26097911e-04,
       3.51887668e-03, 5.72235653e-02, 6.12894188e-02, 3.37191429e-01,
       2.01683575e-01, 1.49025424e-03, 5.02734101e-03, 6.65911287e-02,
       3.64232242e-02, 2.90647715e-02, 6.78156526e-03, 7.41198387e-02,
       7.58455353e-01, 8.74763252e-01, 6.48837381e-02, 9.50352808e-01,
       6.37867133e-03, 7.05344590e-03, 4.07358419e-01, 1.40293371e-02,
       1.35928262e-04, 4.21170895e-02, 3.30484915e-02, 1.16678030e-01,
       1.79142378e-02, 3.49601033e-02, 3.13458081e-03, 8.69456385e-11,
       1.24433049e-02, 1.42861757e-01, 1.80219839e-01, 8.51785502e-03,
       4.04669562e-02, 1.49645988e-02, 1.46989394e-01, 4.20135747e-03,
       9.99997682e-01, 1.13221410e-02, 6.53244826e-04])

In [6]:
def calculate_tau(data, propensity):
    copy = data.copy()
    copy.insert(1, 'Propensity', propensity)
    treatment1 = copy.loc[copy['Treatment'] == 1]
    treatment0 = copy.loc[copy['Treatment'] == 0]
    tau = np.mean(treatment1['CO2 (lb/MWh)'] / treatment1['Propensity']) - np.mean(treatment0['CO2 (lb/MWh)'] / (1 - treatment0['Propensity'])) 
    return tau

def calculate_tau_trimmed(data, propensity):
    copy = data.copy()
    copy.insert(1, 'Propensity', propensity)
    trimmed = copy[(copy['Propensity'] >= 0.1) & (copy['Propensity'] <= 0.9)]
    trimmed1 = trimmed.loc[trimmed['Treatment'] == 1]
    trimmed0 = trimmed.loc[trimmed['Treatment'] == 0]
    tau = np.mean(trimmed1['CO2 (lb/MWh)'] / trimmed1['Propensity']) - np.mean(trimmed0['CO2 (lb/MWh)'] / (1 - trimmed0['Propensity'])) 
    return tau

In [7]:
tau = calculate_tau(total, propensity_scores)
tau

1077.1091590000067

## Trimmed Propensity Scores

In [8]:
tau_trimmed = calculate_tau_trimmed(total, propensity_scores)
tau_trimmed

-687.0153836959075

# Bootstrapping to Construst a 95% Confidence Interval for The True ATE 

In [11]:
def bootstrap_tau(data):
    sample = data.sample(n=len(data), replace=True)
    propensity = calculate_propensity(sample)
    estimated_ate = calculate_tau(sample, propensity)
    return estimated_ate

def get_bootstrap_tau(data, n):
    estimates = []
    for i in np.arange(n):
        estimates.append(bootstrap_tau(data))
    return estimates

ates = get_bootstrap_tau(total, 1000)
confidence_interval = [np.percentile(ates, 2.5), np.percentile(ates, 97.5)]
confidence_interval

[-789.3153338180823, 5686.313140738801]

In [10]:
def bootstrap_tau_trimmed(data):
    sample = data.sample(n=len(data), replace=True)
    propensity = calculate_propensity(sample)
    estimated_ate = calculate_tau_trimmed(sample, propensity)
    
    return estimated_ate

def get_bootstrap_tau_trimmed(data, n):
    estimates = []
    for i in range(n):
        estimate = bootstrap_tau_trimmed(data)
        if not np.isnan(estimate):
            estimates.append(estimate)
    return estimates

ates = get_bootstrap_tau_trimmed(total, 1000)
confidence_interval = [np.percentile(ates, 2.5), np.percentile(ates, 97.5)]
confidence_interval

[-2715.428862424673, 2336.9097629467988]

# PCA to Reduce Confounder Dimensionality

In [11]:
treatment = total['Treatment']
outcome = total['CO2 (lb/MWh)']
confounders.head()

,renew prod (1000MWh),nonrenew prod (1000MWh),production (1000MWh),Urban Population,Urban Population (%),avg annual temp (F),avg annual precip (in),avg annual sunlight (kJ/m^2),num_plants,num_renewable,num_nonrenewable,total area (mile^2),water percentage,total consumption,Renewable Policies
0,8794,4736,13530,526309.0,38.6,41.0,45.5,3815.0,196,170,26,35380,0.128,35399,57
1,8917,17462,26379,6416895.0,91.3,47.9,48.6,3944.0,632,555,78,10554,0.261,131292,193
2,2784,16224,19008,803420.0,58.3,43.8,47.9,3891.0,60,38,22,9349,0.042,33311,76
3,1684,7023,8707,999191.0,91.1,50.1,49.1,3989.0,100,86,14,1545,0.331,53421,61
4,2589,9,2598,225850.0,35.1,42.9,46.0,3826.0,118,105,13,9616,0.042,113,116


In [12]:
scaler = StandardScaler()
confounders_standardized = scaler.fit_transform(confounders)

In [13]:
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(confounders_standardized)

In [14]:
print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Number of components:", pca.n_components_)

Explained variance ratio: [0.48203042 0.15509952 0.14473774 0.07485476 0.05744289 0.02738104
 0.0217565 ]
Number of components: 7


In [15]:
pca_total = pd.DataFrame(X_pca)
pca_total.insert(0, 'Treatment', treatment)
pca_total.insert(1, 'Outcome', outcome)
pca_total.head()

,Treatment,Outcome,0,1,2,3,4,5,6
0,1,336.612,-2.600760,-1.206091,0.397419,-1.863355,0.085522,1.162430,0.013411
1,1,851.739,0.190767,-3.338891,1.653666,1.312415,-0.169352,0.147491,-0.074582
2,1,302.928,-2.615425,-0.654209,0.510820,-1.076607,-0.048116,-0.463036,-0.131896
3,1,811.317,-2.126305,-1.500206,2.043165,1.916484,1.171675,0.479913,0.532047
4,1,35.627,-2.849992,-1.041117,0.350168,-2.163066,-0.568596,0.256658,0.103857


In [16]:
def calculate_propensity_pca(data):
    confounder = data.iloc[:, 1:]
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(confounders, data['Treatment'])
    propensity_scores = log_reg.predict_proba(confounders)[:, 1] 
    return propensity_scores

def calculate_tau_pca(data, propensity):
    copy = data.copy()
    copy.insert(1, 'Propensity', propensity)
    treatment1 = copy.loc[copy['Treatment'] == 1]
    treatment0 = copy.loc[copy['Treatment'] == 0]
    tau = np.mean(treatment1['Outcome'] / treatment1['Propensity']) - np.mean(treatment0['Outcome'] / (1 - treatment0['Propensity'])) 
    return tau

In [17]:
pca_propensity = calculate_propensity_pca(pca_total)
pca_tau = calculate_tau_pca(pca_total, pca_propensity)
pca_tau

1077.1091590000067

# Non-parametric Method to Calculate Propensity Scores

In [18]:
X = total.iloc[:, 4:]
T = total['Treatment']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, T_train, T_test = train_test_split(X_scaled, T, test_size=0.2)

T_train = np.array(T_train)
T_test = np.array(T_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
T_train_tensor = torch.tensor(T_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
T_test_tensor = torch.tensor(T_test, dtype=torch.float32).unsqueeze(1)

class PropensityNet(nn.Module):
    def __init__(self, input_dim):
        super(PropensityNet, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),                 
            nn.Dropout(0.4),            
            nn.Linear(256, 64),          
            nn.ReLU(),                  
            nn.BatchNorm1d(64),
            nn.Linear(64, 32),   
            nn.ReLU(),                  
            nn.Dropout(0.3),            
            nn.Linear(32, 16),
            nn.ReLU(),                  
            nn.BatchNorm1d(16),           
            nn.Linear(16, 8),
            nn.ReLU(),                  
            nn.Dropout(0.2),            
            nn.Linear(8, 1),  
            nn.Sigmoid()                
        )
    
    def forward(self, x):
        return self.network(x)

input_dim = X_train_tensor.shape[1]
model = PropensityNet(input_dim)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

# Train the Model
num_epochs = 100
batch_size = 16

train_dataset = torch.utils.data.TensorDataset(X_train_tensor, T_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    correct_preds = 0
    total_preds = 0
    
    for batch_X, batch_T in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_T)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Classify as 1 if score >= 0.5, else 0
        predictions = (outputs >= 0.5).float()  
        correct_preds += (predictions == batch_T).sum().item()
        total_preds += batch_T.size(0)

        epoch_loss += loss.item()

    accuracy = correct_preds / total_preds  # Calculate accuracy

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_loader):.4f}, Accuracy: {accuracy:.4f}")

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

model.eval()
with torch.no_grad():
    propensity_scores_nn = model(X_tensor).numpy()

Epoch [10/100], Loss: 0.4444, Accuracy: 0.9189
Epoch [20/100], Loss: 0.1080, Accuracy: 0.9459
Epoch [30/100], Loss: 0.1904, Accuracy: 0.9189
Epoch [40/100], Loss: 0.2110, Accuracy: 0.8649
Epoch [50/100], Loss: 0.2394, Accuracy: 0.9189
Epoch [60/100], Loss: 0.1396, Accuracy: 0.9730
Epoch [70/100], Loss: 0.0903, Accuracy: 0.9730
Epoch [80/100], Loss: 0.0661, Accuracy: 0.9730
Epoch [90/100], Loss: 0.0281, Accuracy: 1.0000
Epoch [100/100], Loss: 0.0258, Accuracy: 1.0000


In [19]:
tau_nn = calculate_tau(total, propensity_scores_nn)
tau_nn

-275.26326349264343

In [27]:
scaler = StandardScaler()

total_standardized = pd.DataFrame(scaler.fit_transform(total.iloc[:, 1:]), columns=total.columns[1:])

total_standardized = total_standardized.drop(columns=["Treatment"])
total_standardized.insert(0, 'State', total['State'])

total_standardized.head()

,State,Electricity Price (cents/kWh),CO2 (lb/MWh),renew prod (1000MWh),nonrenew prod (1000MWh),production (1000MWh),Urban Population,Urban Population (%),avg annual temp (F),avg annual precip (in),avg annual sunlight (kJ/m^2),num_plants,num_renewable,num_nonrenewable,total area (mile^2),water percentage,total consumption,Renewable Policies
0,Maine,1.553289,-1.176558,-0.445166,-0.913296,-0.866537,-0.729234,-2.255355,-1.463139,0.552644,-1.232236,-0.224137,-0.059037,-0.555084,-0.659464,0.587026,-0.638086,-0.756434
1,Massachusetts,2.655626,0.034395,-0.441370,-0.734036,-0.726064,0.130312,1.302337,-0.558648,0.775260,-0.969162,1.165162,1.597209,-0.046985,-1.189422,2.033988,-0.402855,1.270022
2,New Hampshire,2.598062,-1.255742,-0.630639,-0.751475,-0.806648,-0.688798,-0.925439,-1.096099,0.724992,-1.077246,-0.657497,-0.626893,-0.594168,-1.215145,-0.348604,-0.643208,-0.473326
3,Rhode Island,2.088628,-0.060628,-0.664586,-0.881081,-0.919265,-0.660232,1.288836,-0.270260,0.811166,-0.877391,-0.530038,-0.420400,-0.672337,-1.381737,2.795547,-0.593877,-0.696832
4,Vermont,1.423772,-1.884109,-0.636657,-0.979881,-0.986052,-0.773077,-2.491634,-1.214076,0.588550,-1.209803,-0.472682,-0.338663,-0.682108,-1.209446,-0.348604,-0.724645,0.122691


In [47]:
#indeces = [1] + list(range(3, 1)) + list(range(12, len(total_standardized.columns)))

indices = [1] + list(range(3, len(total_standardized.columns)))
X = total_standardized.iloc[:, indices]
y = total_standardized['CO2 (lb/MWh)']

glm_model = sm.GLM(y, sm.add_constant(X), family=sm.families.Gaussian())

glm_results = glm_model.fit()

print(glm_results.summary())
odds_ratios = np.exp(glm_results.params)
print("\nOdds Ratios:")
print(odds_ratios)

                 Generalized Linear Model Regression Results                  
Dep. Variable:           CO2 (lb/MWh)   No. Observations:                   47
Model:                            GLM   Df Residuals:                       31
Model Family:                Gaussian   Df Model:                           15
Link Function:               Identity   Scale:                          1.0207
Method:                          IRLS   Log-Likelihood:                -57.392
Date:                Tue, 10 Dec 2024   Deviance:                       31.643
Time:                        21:09:42   Pearson chi2:                     31.6
No. Iterations:                     3   Pseudo R-squ. (CS):             0.3269
Covariance Type:            nonrobust                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         